In [3]:
import time

import pandas as pd
import shap
import numpy as np
import tensorflow as tf
from pathlib import Path

from blume.table import table
from scipy.cluster.vq import whiten
from tqdm import tqdm
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt
from data_processing import pre_process_data
from obfuscation_functions import *
from util.custom_functions import replace_outliers_by_std, mean_std_analysis, replace_outliers_by_quartile
from shap_experiment import extract_shap, extract_shap_values, parse_shap_values_by_class


Num GPUs Available:  0


In [4]:
# datasets
audio_files_path = "./NNDatasets/audio"
gen_shap_df_path = './data/gen_shap_df.npy'
emo_shap_df_path = './data/emo_shap_df.npy'
print("Pre-processing audio files!")
x_train_emo_cnn, y_train_emo_encoded, x_test_emo_cnn, y_test_emo_encoded = pre_process_data(audio_files_path, get_emotion_label=True)
x_train_gen_cnn, y_train_gen_encoded, x_test_gen_cnn, y_test_gen_encoded = pre_process_data(audio_files_path, get_emotion_label=False)
print("Pre-processing audio files Complete!")

Pre-processing audio files!


100%|███████████████████████████████████████████| 6507/6507 [00:00<00:00, 2524029.98it/s]

Loading pre extracted raw audio from pkl file.


Loading pre extracted raw audio from pkl file successful.
Extracting mfccs from raw nd train audio data split.
Train data found. Loading from npy file.
Loading train successful.
Extracting mfccs from raw nd test audio data split.
Test data found. Loading from npy file.
Loading test data successful.


100%|███████████████████████████████████████████| 6507/6507 [00:00<00:00, 3499914.87it/s]


Loading pre extracted raw audio from pkl file.
Loading pre extracted raw audio from pkl file successful.
Extracting mfccs from raw nd train audio data split.
Train data found. Loading from npy file.
Loading train successful.
Extracting mfccs from raw nd test audio data split.
Test data found. Loading from npy file.
Loading test data successful.
Pre-processing audio files Complete!


In [23]:
# from keras import Sequential
from keras.layers import Conv2D, MaxPool2D
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten, Dropout, Activation
from tensorflow.keras.layers import Conv1D, MaxPooling1D

model = Sequential()
model.add(Dense(128, input_shape=(40, )))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(96))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(32))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(7))
model.add(Activation('softmax'))
opt = optimizers.Adam()
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
model.summary()



Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_17 (Dense)             (None, 128)               5248      
_________________________________________________________________
activation_23 (Activation)   (None, 128)               0         
_________________________________________________________________
dropout_18 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_18 (Dense)             (None, 96)                12384     
_________________________________________________________________
activation_24 (Activation)   (None, 96)                0         
_________________________________________________________________
dropout_19 (Dropout)         (None, 96)                0         
_________________________________________________________________
dense_19 (Dense)             (None, 64)              

In [24]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train_emo_cnn = np.reshape(x_train_emo_cnn, (x_train_emo_cnn.shape[0], x_train_emo_cnn.shape[1]))
x_test_emo_cnn = np.reshape(x_test_emo_cnn, (x_test_emo_cnn.shape[0], x_test_emo_cnn.shape[1]))

x_train_emo_cnn_scaled = sc.fit_transform(x_train_emo_cnn)
x_test_emo_cnn_scaled = sc.transform(x_test_emo_cnn)

emo_train_dataset = tf.data.Dataset.from_tensor_slices((x_train_emo_cnn_scaled, y_train_emo_encoded))
gen_train_dataset = tf.data.Dataset.from_tensor_slices((x_train_emo_cnn_scaled, y_train_gen_encoded))

emo_test_dataset = tf.data.Dataset.from_tensor_slices((x_test_emo_cnn_scaled, y_test_emo_encoded))
gen_test_dataset = tf.data.Dataset.from_tensor_slices((x_test_gen_cnn, y_test_gen_encoded))

In [25]:
cnnhistory = model.fit(x_train_emo_cnn_scaled, y_train_emo_encoded, batch_size=4, epochs=200, validation_data=(x_test_emo_cnn_scaled, y_test_emo_encoded))
# Save the weights
#model.save(model_path)
    

Epoch 1/200
1135/1135 [==============================] - 1s 803us/step - loss: 1.8140 - accuracy: 0.2554 - val_loss: 1.3766 - val_accuracy: 0.4432
Epoch 2/200
1135/1135 [==============================] - 1s 725us/step - loss: 1.4350 - accuracy: 0.4267 - val_loss: 1.2698 - val_accuracy: 0.5075
Epoch 3/200
1135/1135 [==============================] - 1s 724us/step - loss: 1.2890 - accuracy: 0.5033 - val_loss: 1.1680 - val_accuracy: 0.5383
Epoch 4/200
1135/1135 [==============================] - 1s 722us/step - loss: 1.2330 - accuracy: 0.5388 - val_loss: 1.0603 - val_accuracy: 0.6141
Epoch 5/200
1135/1135 [==============================] - 1s 729us/step - loss: 1.1568 - accuracy: 0.5573 - val_loss: 1.0277 - val_accuracy: 0.6088
Epoch 6/200
1135/1135 [==============================] - 1s 730us/step - loss: 1.0291 - accuracy: 0.6200 - val_loss: 0.9786 - val_accuracy: 0.6405
Epoch 7/200
1135/1135 [==============================] - 1s 732us/step - loss: 0.9714 - accuracy: 0.6414 - val_loss: 0

1135/1135 [==============================] - 1s 747us/step - loss: 0.3888 - accuracy: 0.8709 - val_loss: 0.4650 - val_accuracy: 0.8467
Epoch 57/200
1135/1135 [==============================] - 1s 736us/step - loss: 0.3893 - accuracy: 0.8578 - val_loss: 0.4564 - val_accuracy: 0.8485
Epoch 58/200
1135/1135 [==============================] - 1s 731us/step - loss: 0.3724 - accuracy: 0.8758 - val_loss: 0.4668 - val_accuracy: 0.8441
Epoch 59/200
1135/1135 [==============================] - 1s 727us/step - loss: 0.3938 - accuracy: 0.8650 - val_loss: 0.4497 - val_accuracy: 0.8388
Epoch 60/200
1135/1135 [==============================] - 1s 743us/step - loss: 0.4003 - accuracy: 0.8628 - val_loss: 0.4539 - val_accuracy: 0.8529
Epoch 61/200
1135/1135 [==============================] - 1s 747us/step - loss: 0.3910 - accuracy: 0.8636 - val_loss: 0.4640 - val_accuracy: 0.8485
Epoch 62/200
1135/1135 [==============================] - 1s 745us/step - loss: 0.3716 - accuracy: 0.8767 - val_loss: 0.4561 

1135/1135 [==============================] - 1s 749us/step - loss: 0.3098 - accuracy: 0.9031 - val_loss: 0.4052 - val_accuracy: 0.8546
Epoch 112/200
1135/1135 [==============================] - 1s 743us/step - loss: 0.2850 - accuracy: 0.9076 - val_loss: 0.4422 - val_accuracy: 0.8590
Epoch 113/200
1135/1135 [==============================] - 1s 748us/step - loss: 0.2831 - accuracy: 0.9084 - val_loss: 0.4242 - val_accuracy: 0.8705
Epoch 114/200
1135/1135 [==============================] - 1s 765us/step - loss: 0.2920 - accuracy: 0.9048 - val_loss: 0.4733 - val_accuracy: 0.8652
Epoch 115/200
1135/1135 [==============================] - 1s 768us/step - loss: 0.2659 - accuracy: 0.9118 - val_loss: 0.3932 - val_accuracy: 0.8696
Epoch 116/200
1135/1135 [==============================] - 1s 746us/step - loss: 0.3006 - accuracy: 0.9009 - val_loss: 0.4170 - val_accuracy: 0.8740
Epoch 117/200
1135/1135 [==============================] - 1s 745us/step - loss: 0.2892 - accuracy: 0.9031 - val_loss: 0

1135/1135 [==============================] - 1s 746us/step - loss: 0.2537 - accuracy: 0.9228 - val_loss: 0.4438 - val_accuracy: 0.8934
Epoch 167/200
1135/1135 [==============================] - 1s 743us/step - loss: 0.2670 - accuracy: 0.9193 - val_loss: 0.4364 - val_accuracy: 0.8952
Epoch 168/200
1135/1135 [==============================] - 1s 744us/step - loss: 0.2431 - accuracy: 0.9185 - val_loss: 0.4361 - val_accuracy: 0.8881
Epoch 169/200
1135/1135 [==============================] - 1s 753us/step - loss: 0.2598 - accuracy: 0.9182 - val_loss: 0.4674 - val_accuracy: 0.8714
Epoch 170/200
1135/1135 [==============================] - 1s 768us/step - loss: 0.2476 - accuracy: 0.9251 - val_loss: 0.4138 - val_accuracy: 0.8837
Epoch 171/200
1135/1135 [==============================] - 1s 745us/step - loss: 0.2389 - accuracy: 0.9268 - val_loss: 0.4427 - val_accuracy: 0.8740
Epoch 172/200
1135/1135 [==============================] - 1s 742us/step - loss: 0.2788 - accuracy: 0.9137 - val_loss: 0

In [26]:
model.evaluate(x_test_emo_cnn_scaled, y_test_emo_encoded)

36/36 [==============================] - 0s 464us/step - loss: 0.4091 - accuracy: 0.8899


[0.4091288149356842, 0.8898678421974182]

In [28]:
model.save("emo_checkpoint/emo_model_simple.h5")

In [60]:
t = tf.constant([[1, 1 , 1], [1, 1, 1]])
paddings = tf.constant([[0, 0], [0, 20]])
tf.print(t)

[[1 1 1]
 [1 1 1]]


In [61]:
t.shape

TensorShape([2, 3])

In [62]:
tf.pad(t, paddings)

<tf.Tensor: shape=(2, 23), dtype=int32, numpy=
array([[1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0],
       [1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0]], dtype=int32)>

In [37]:
x

array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.]])